In [194]:
import numpy as np
import sys
from __future__ import division #to have float division

In [195]:
# We have an entity-relation graph modelled after a set of entity-relation triplets (e_1, r, e_2).
# We have N nodes, N_e entity nodes and N_r relation nodes.
# We have two 2d lists of communities, where each row is a community that contains the nodes in that community.
# One matrix is the K_e matrix for entity communities, and one is the K_r matrix for relation communities.

def initialize():
    numEntityNodes = 6;
    numRelationNodes = 5;
    numTotalNodes = numEntityNodes + numRelationNodes;
    outboundAdjacencyMatrix = np.zeros((numEntityNodes, numRelationNodes)) # cell (i,j) = 1 if edge, 0 otherwise
    inboundAdjacencyMatrix = np.zeros((numEntityNodes, numRelationNodes)) # cell (i,j) = 1 if edge, 0 otherwise

    K_entity = 2;
    K_relation = 2;
    
    # cell (i,j) = 1 if node j belongs to community i. It is 0 else
    commEntityMatrix = np.zeros((K_entity, numEntityNodes)) #Matrix of entity communities.
    commRelationMatrix = np.zeros((K_relation, numRelationNodes)) #Matrix of relation communities
    
    commsByEntity = np.zeros(K_entity) #Index i of this array stores the community that entity i belongs to
    commsByRelation = np.zeros(K_relation) #Index i of this array stores the community that relation i belongs to
    
    entityCommOutboundEdges = np.zeros((K_entity, K_relation))
    entityCommInboundEdges = np.zeros((K_entity, K_relation))
    relationCommOutboundEdges = np.zeros((K_relation, K_entity))
    relationCommInboundEdges = np.zeros((K_relation, K_entity))
    
    # Randomly assign a node to one of the communities
    # FOR TESTING ONLY
    commsByEntity = np.array([0,0,1,0,0,1])
    commsByRelation = np.array([0,0,1,0,1])
#     commsByEntity = np.random.randint(0, K_entity, numEntityNodes)
#     commsByRelation = np.random.randint(0, K_relation, numRelationNodes)
    
    # FOR TESTING ONLY FOR TESTING
    outboundAdjacencyMatrix = np.array([[1,0,0,0,0],[0,1,0,0,0],[0,0,1,0,0],[0,0,0,1,0],[0,0,0,0,1],[0,0,0,0,0]])
    inboundAdjacencyMatrix = np.array([[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],[1,0,0,0,0],[0,1,0,1,0],[0,0,1,0,1]])

    #CREATE THE COMMUNITY MATRICES
    for entityNode in xrange(numEntityNodes):
        commEntityMatrix[commsByEntity[entityNode]][entityNode] = 1
    for relationNode in xrange(numRelationNodes):
        commRelationMatrix[commsByRelation[relationNode]][relationNode] = 1
    
    # Store the inboundEdges and outboundEdges for each community
    for entityNode in xrange(numEntityNodes):
        entityComm = commsByEntity[entityNode]
        curOutboundEdges = outboundAdjacencyMatrix[entityNode]
        curInboundEdges = inboundAdjacencyMatrix[entityNode]
        for relationNode in xrange(numRelationNodes):
            relComm = commsByRelation[relationNode]
            
            # If there is an outbound edge to that relation node, update the outbound/inbound edges on both sides
            if curOutboundEdges[relationNode] == 1:
                entityCommOutboundEdges[entityComm][relComm] += 1
                relationCommInboundEdges[relComm][entityComm] += 1
            if curInboundEdges[relationNode] == 1:
                entityCommInboundEdges[entityComm][relComm] += 1
                relationCommOutboundEdges[relComm][entityComm] += 1
    
    return [numEntityNodes, numRelationNodes, numTotalNodes, outboundAdjacencyMatrix, inboundAdjacencyMatrix,
           K_entity, K_relation, commEntityMatrix, commRelationMatrix, entityCommOutboundEdges, 
           entityCommInboundEdges, relationCommOutboundEdges, relationCommInboundEdges,
           commsByEntity, commsByRelation]

In [196]:
# Randomize initial communities
def initializeAssignments():
    # Randomly assign a node to one of the communities
    commsByEntity = np.random.randint(0, K_entity, numEntityNodes)
    commsByRelation = np.random.randint(0, K_relation, numRelationNodes)
    
    # Store the inboundEdges and outboundEdges for each community
    for entityNode in xrange(numEntityNodes):
        entityComm = commsByEntity[entityNode]
        curOutboundEdges = outboundAdjacencyMatrix[entityNode]
        curInboundEdges = inboundAdjacencyMatrix[entityNode]
        for relationNode in xrange(numRelationNodes):
            relComm = commsByRelation[relationNode]
            
            # If there is an outbound edge to that relation node, update the outbound/inbound edges on both sides
            if curOutboundEdges[relationNode] == 1:
                entityCommOutboundEdges[entityComm][relComm] += 1
                relationCommInboundEdges[relComm][entityComm] += 1
            if curInboundEdges[relationNode] == 1:
                entityCommInboundEdges[entityComm][relComm] += 1
                relationCommOutboundEdges[relComm][entityComm] += 1

In [204]:
# Define the algorithm for one iteration
def algoIteration(numEntityNodes, numRelationNodes, numTotalNodes, outboundAdjacencyMatrix, inboundAdjacencyMatrix, 
               K_entity, K_relation, commEntityMatrix, commRelationMatrix, entityCommOutboundEdges,
               entityCommInboundEdges, relationCommOutboundEdges, relationCommInboundEdges,commsByEntity, 
               commsByRelation, bestStateCommsByEntity, bestStateCommsByRelation, bestStateTotalPenalty):
    
    
    
    # calculate entity node penalties
    totalEntityPenalty = 0
    maxEntityPenalty = -sys.maxint - 1
    entityNodeToChange = -1
    changeEntityNodeOutboundEdges = np.zeros(K_relation)
    changeEntityNodeInboundEdges = np.zeros(K_relation)
    
    # Calculate outbound edges and inbound edges for this node
    for entityNode in xrange(numEntityNodes):
        print "curEntityNode: ", entityNode
        
        curNodeOutboundEdges = np.zeros(K_relation)
        curNodeInboundEdges = np.zeros(K_relation)
        
        # We calculate its outbound and inbound edges to all relation communities
        outboundEdges = outboundAdjacencyMatrix[entityNode]
        for relationNode in xrange(numRelationNodes):
            outboundComm = commsByRelation[relationNode]
            if outboundEdges[relationNode] == 1:
                curNodeOutboundEdges[outboundComm] += 1
        
        inboundEdges = inboundAdjacencyMatrix[entityNode]
        for relationNode in xrange(numRelationNodes):
            inboundComm = commsByRelation[relationNode]
            if inboundEdges[relationNode] == 1:
                curNodeInboundEdges[inboundComm] += 1
            
        print "outboundEdges: ", curNodeOutboundEdges
        print "inboundEdges: ", curNodeInboundEdges
        
        # Calculate the penalty for this entity node
        # Its current community is stored in commsByEntity
        curComm = commsByEntity[entityNode]
        curCommOutboundEdges = entityCommOutboundEdges[curComm]
        curCommInboundEdges = entityCommInboundEdges[curComm]
        numNodesInComm = np.sum(commEntityMatrix[curComm])
        print "curComm: ", curComm
        print "curCommOutboundEdges: ", curCommOutboundEdges
        print "curCommInboundEdges: ", curCommInboundEdges
        print "numNodesInComm: ", numNodesInComm
        
        # The penalty is the sum of squared differences between the mean outbound of this community and
        # current node outbound plus mean inbound minus current node inbound
        outboundDiff = np.sum(np.square(curCommOutboundEdges/numNodesInComm - curNodeOutboundEdges))
        inboundDiff = np.sum(np.square(curCommInboundEdges/numNodesInComm - curNodeInboundEdges))
        print "outboundDiff: ", outboundDiff
        print "inboundDiff: ", inboundDiff
        curPenalty = outboundDiff + inboundDiff
        
        print "curPenalty", curPenalty
        totalEntityPenalty += curPenalty
        
        if curPenalty > maxEntityPenalty:
            maxEntityPenalty = curPenalty
            entityNodeToChange = entityNode
            changeEntityNodeOutboundEdges = curNodeOutboundEdges
            changeEntityNodeInboundEdges = curNodeInboundEdges
            
    # Figure out where to change the node, but DON'T change it yet
    
    print "maxPenalty ", maxEntityPenalty
    print "entityNodeToChange ", entityNodeToChange
    
    newCommForEntityBoolean = False
    minEntityPenalty = sys.maxint
    newCommForEntity = -1
    for curComm in xrange(K_entity):
        curCommOutboundEdges = entityCommOutboundEdges[curComm]
        curCommInboundEdges = entityCommInboundEdges[curComm]
        numNodesInComm = np.sum(commEntityMatrix[curComm])
        print "curComm: ", curComm
        print "curCommOutboundEdges: ", curCommOutboundEdges
        print "curCommInboundEdges: ", curCommInboundEdges
        print "numNodesInComm: ", numNodesInComm
        
        # The penalty is the sum of squared differences between the mean outbound of this community and
        # current node outbound plus mean inbound minus current node inbound
        outboundDiff = np.sum(np.square(curCommOutboundEdges/numNodesInComm - changeEntityNodeOutboundEdges))
        inboundDiff = np.sum(np.square(curCommInboundEdges/numNodesInComm - changeEntityNodeInboundEdges))
        print "outboundDiffMoving: ", outboundDiff
        print "inboundDiffMoving: ", inboundDiff
        print "curPenaltyMoving, ", (outboundDiff + inboundDiff)
        
        if (outboundDiff + inboundDiff) < minEntityPenalty:
            minEntityPenalty = outboundDiff + inboundDiff
            newCommForEntity = curComm
    # Check if it is actually a new community for this entity node
    if not newCommForEntity == commsByEntity[entityNodeToChange]:
        newCommForEntityBoolean = True
        
    print "minPenalty ", minEntityPenalty
    print "newCommForEntity, ", newCommForEntity
    print "newCommForEntityBoolean: ", newCommForEntityBoolean
    
    print ""
    print ""
    print ""
    
    # calculate relation node penalties
    totalRelationPenalty = 0
    maxRelationPenalty = -sys.maxint - 1
    relationNodeToChange = -1
    changeRelationNodeOutboundEdges = np.zeros(K_entity)
    changeRelationNodeInboundEdges = np.zeros(K_entity)
    
    # Calculate outbound edges and inbound edges for this node
    for relationNode in xrange(numRelationNodes):     
        print "curRelationNode: ", relationNode
        curNodeOutboundEdges = np.zeros(K_entity)
        curNodeInboundEdges = np.zeros(K_entity)
        
        # We calculate its outbound and inbound edges to all relation communities
        # Note that the outbound for a relation is the inbound for an entity
        outboundEdges = inboundAdjacencyMatrix.T[relationNode]
        for entityNode in xrange(numEntityNodes):
            outboundComm = commsByEntity[entityNode]
            if outboundEdges[entityNode] == 1:
                curNodeOutboundEdges[outboundComm] += 1
        
        inboundEdges = outboundAdjacencyMatrix.T[relationNode]
        for entityNode in xrange(numEntityNodes):
            inboundComm = commsByEntity[relationNode]
            if inboundEdges[entityNode] == 1:
                curNodeInboundEdges[inboundComm] += 1
        
        print "outboundEdges: ", curNodeOutboundEdges
        print "inboundEdges: ", curNodeInboundEdges
        
        # Calculate the penalty for this entity node
        # Its current community is stored in commsByEntity
        curComm = commsByRelation[relationNode]
        curCommOutboundEdges = relationCommOutboundEdges[curComm]
        curCommInboundEdges = relationCommInboundEdges[curComm]
        numNodesInComm = np.sum(commRelationMatrix[curComm])
        print "curComm: ", curComm
        print "curCommOutboundEdges: ", curCommOutboundEdges
        print "curCommInboundEdges: ", curCommInboundEdges
        print "numNodesInComm: ", numNodesInComm
        
        # The penalty is the sum of squared differences between the mean outbound of this community and
        # current node outbound plus mean inbound minus current node inbound
        outboundDiff = np.sum(np.square(curCommOutboundEdges/numNodesInComm - curNodeOutboundEdges))
        inboundDiff = np.sum(np.square(curCommInboundEdges/numNodesInComm - curNodeInboundEdges))
        print "outboundDiff: ", outboundDiff
        print "inboundDiff: ", inboundDiff
        curPenalty = outboundDiff + inboundDiff
        print "curPenalty", curPenalty
        totalRelationPenalty += curPenalty
        
        if curPenalty > maxRelationPenalty:
            maxRelationPenalty = curPenalty
            relationNodeToChange = relationNode
            changeRelationNodeOutboundEdges = curNodeOutboundEdges
            changeRelationNodeInboundEdges = curNodeInboundEdges
          
    print "maxPenalty ", maxRelationPenalty
    print "relationNodeToChange ", relationNodeToChange
    
    # Figure out where to move the worst relation node
    newCommForRelationBoolean = False
    minRelationPenalty = sys.maxint
    newCommForRelation = -1
    for curComm in xrange(K_relation):
        curCommOutboundEdges = relationCommOutboundEdges[curComm]
        currCommInboundEdges = relationCommInboundEdges[curComm]
        numNodesInComm = np.sum(commRelationMatrix[curComm])
        print "curComm: ", curComm
        print "curCommOutboundEdges: ", curCommOutboundEdges
        print "curCommInboundEdges: ", curCommInboundEdges
        print "numNodesInComm: ", numNodesInComm
        
        # The penalty is the sum of squared differences between the mean outbound of this community and
        # current node outbound plus mean inbound minus current node inbound
        outboundDiff = np.sum(np.square(curCommOutboundEdges/numNodesInComm - changeRelationNodeOutboundEdges))
        inboundDiff = np.sum(np.square(curCommInboundEdges/numNodesInComm - changeRelationNodeInboundEdges))
        print "outboundDiffMoving: ", outboundDiff
        print "inboundDiffMoving: ", inboundDiff
        print "curPenaltyMoving, ", (outboundDiff + inboundDiff)
        if (outboundDiff + inboundDiff) < minRelationPenalty:
            minRelationPenalty = outboundDiff + inboundDiff
            newCommForRelation = curComm
    # Check if it is actually a new community for this entity node
    if not newCommForRelation == commsByRelation[relationNodeToChange]:
        newCommForRelationBoolean = True
    
    print "minPenalty ", minRelationPenalty
    print "newCommForRelation, ", newCommForRelation
    print "newCommForrelationBoolean: ", newCommForRelationBoolean
        
    if not newCommForEntityBoolean and not newCommForRelationBoolean:
        # We might get stuck in local minima, so move if our penalties are greater than the tolerance
        # Move to a random community
        print "local minimum"
        tolerance = 0.1
        if maxEntityPenalty > tolerance:
            newCommForEntity = np.random.randint(0,K_entity)
            newCommForEntityBoolean = True
        if maxRelationPenalty > tolerance:
            newCommForRelation = np.random.randint(0, K_relation)
            newCommForRelationBoolean = True
            
        print "newCommForEntity: ", newCommForEntity
        print "newCommForEntityBoolean: ", newCommForEntityBoolean
        print "newCommForRelation: ", newCommForRelation
        print "newCommForRelationBoolean: ", newCommForRelationBoolean
            
        if not newCommForEntityBoolean and not newCommForRelationBoolean:
            return False
    
    
    print ""
    print ""
    print ""
        
    # Move the entity node and relation node to their new communities
    if newCommForEntityBoolean:
        # Move entity node first
        entityPrevComm = commsByEntity[entityNodeToChange]
        # Update the inbound/outbound edges on the entity side by subtracting from current comm and adding to new comm
        entityCommOutboundEdges[entityPrevComm] -=  changeEntityNodeOutboundEdges
        entityCommInboundEdges[entityPrevComm] -= changeEntityNodeInboundEdges
        entityCommOutboundEdges[newCommForEntity] += changeEntityNodeOutboundEdges
        entityCommInboundEdges[newCommForEntity] += changeEntityNodeInboundEdges

        # Update the inbound/outbound edges on the relation side that are changed by moving this entity node
        outboundLinks = outboundAdjacencyMatrix[entityNodeToChange]
        inboundLinks = inboundAdjacencyMatrix[entityNodeToChange]
        for relationNode in xrange(numRelationNodes):
            relComm = commsByRelation[relationNode]

            # If there is an outbound edge from entity to relation, change relation inbound edges and vice versa
            if outboundLinks[relationNode] == 1:
                relationCommInboundEdges[relComm][entityPrevComm] -= 1
                relationCommInboundEdges[relComm][newCommForEntity] += 1
            if inboundLinks[relationNode] == 1:
                relationCommOutboundEdges[relComm][entityPrevComm] -= 1
                relationCommOutboundEdges[relComm][newCommForEntity] += 1
        # Update the actual community matrix
        commEntityMatrix[entityPrevComm][entityNodeToChange] = 0
        commEntityMatrix[newCommForEntity][entityNodeToChange] = 1
    
    if newCommForRelationBoolean:
        # Move relation node
        relationPrevComm = commsByRelation[relationNodeToChange]
        # Update the inbound/outbound edges on the relation side by subtracting from current comm and adding to new comm
        relationCommOutboundEdges[relationPrevComm] -= changeRelationNodeOutboundEdges
        relationCommInboundEdges[relationPrevComm] -= changeRelationNodeInboundEdges
        relationCommOutboundEdges[newCommForRelation] += changeRelationNodeOutboundEdges
        relationCommInboundEdges[newCommForRelation] += changeRelationNodeInboundEdges
        
        # Update the inbound/outbound edges on the entity side that are changed by moving this relation node
        outboundLinks = outboundAdjacencyMatrix.T[relationNodeToChange]
        inboundLinks = inboundAdjacencyMatrix.T[relationNodeToChange]
        for entityNode in xrange(numEntityNodes):
            entityComm = commsByEntity[entityNode]
            
            if outboundLinks[entityNode] == 1:
                entityCommInboundEdges[entityComm][relationPrevComm] -= 1
                entityCommInboundEdges[entityComm][newCommForRelation] += 1
            if inboundLinks[entityNode] == 1:
                entityCommOutboundEdges[entityComm][relationPrevComm] -= 1
                entityCommOutboundEdges[entityComm][newCommForRelation] += 1
        # Update the actual community matrix
        commRelationMatrix[relationPrevComm][relationNodeToChange] = 0
        commRelationMatrix[newCommForRelation][relationNodeToChange] = 1
    
    commsByEntity[entityNodeToChange] = newCommForEntity
    commsByRelation[relationNodeToChange] = newCommForRelation
    print "NEW COMMS BY ENTITY: ", commsByEntity
    print "NEW COMMS BY RELATION: ", commsByRelation
    
    totalPenalty = totalEntityPenalty - maxEntityPenalty + minEntityPenalty + totalRelationPenalty - maxRelationPenalty + minRelationPenalty
    if totalPenalty < bestStateTotalPenalty:
        bestStateCommsByEntity = commsByEntity
        bestStateCommsByRelation = commsByRelation
        bestStateTotalPenalty = totalPenalty
    
    return True

In [205]:
if __name__ == '__main__':
    [numEntityNodes, numRelationNodes, numTotalNodes, outboundAdjacencyMatrix, inboundAdjacencyMatrix, 
               K_entity, K_relation, commEntityMatrix, commRelationMatrix, entityCommOutboundEdges,
               entityCommInboundEdges, relationCommOutboundEdges, relationCommInboundEdges,commsByEntity, 
               commsByRelation] = initialize()

In [206]:
    testOutbound = np.random.randint(0, 2, (numEntityNodes, numRelationNodes))
    testInbound = np.random.randint(0,2,(numEntityNodes, numRelationNodes))
    
#     outboundAdjacencyMatrix = testOutbound
#     # inbound is just the flipped of outbound
#     flipBinary = np.vectorize(lambda a: 1-a)
#     inboundAdjacencyMatrix = flipBinary(outboundAdjacencyMatrix)
    
    print "numEntityNodes: ",numEntityNodes
    print "numRelationNodes: ",numRelationNodes
    print "numTotalNodes: ",numTotalNodes
    print "outboundAdjacencyMatrix: ",outboundAdjacencyMatrix
    print "inboundAdjacencyMatrix: ",inboundAdjacencyMatrix
    print "K_entity: ",K_entity
    print "K_relation: ",K_relation
    print "commEntityMatrix: ",commEntityMatrix
    print "commRelationMatrix: ",commRelationMatrix
    print "entityCommOutboundEdges: ",entityCommOutboundEdges
    print "entityCommInboundEdges: ",entityCommInboundEdges
    print "relationCommOutboundEdges: ",relationCommOutboundEdges
    print "relationCommInboundEdges: ",relationCommInboundEdges
    print "commsByEntity: ",commsByEntity
    print "commsByRelation: ",commsByRelation

numEntityNodes:  6
numRelationNodes:  5
numTotalNodes:  11
outboundAdjacencyMatrix:  [[1 0 0 0 0]
 [0 1 0 0 0]
 [0 0 1 0 0]
 [0 0 0 1 0]
 [0 0 0 0 1]
 [0 0 0 0 0]]
inboundAdjacencyMatrix:  [[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [1 0 0 0 0]
 [0 1 0 1 0]
 [0 0 1 0 1]]
K_entity:  2
K_relation:  2
commEntityMatrix:  [[ 1.  1.  0.  1.  1.  0.]
 [ 0.  0.  1.  0.  0.  1.]]
commRelationMatrix:  [[ 1.  1.  0.  1.  0.]
 [ 0.  0.  1.  0.  1.]]
entityCommOutboundEdges:  [[ 3.  1.]
 [ 0.  1.]]
entityCommInboundEdges:  [[ 3.  0.]
 [ 0.  2.]]
relationCommOutboundEdges:  [[ 3.  0.]
 [ 0.  2.]]
relationCommInboundEdges:  [[ 3.  0.]
 [ 1.  1.]]
commsByEntity:  [0 0 1 0 0 1]
commsByRelation:  [0 0 1 0 1]


In [207]:
    bestStateTotalPenalty = sys.maxint
    bestStateCommsByEntity = commsByEntity
    bestStateCommsByRelation = commsByRelation
    
    for i in xrange(5):
        print algoIteration(numEntityNodes, numRelationNodes, numTotalNodes, outboundAdjacencyMatrix, inboundAdjacencyMatrix, 
                   K_entity, K_relation, commEntityMatrix, commRelationMatrix, entityCommOutboundEdges,
                   entityCommInboundEdges, relationCommOutboundEdges, relationCommInboundEdges,commsByEntity, 
                   commsByRelation, bestStateCommsByEntity, bestStateCommsByRelation, bestStateTotalPenalty)
        print "commEntityMatrix: ",commEntityMatrix
        print "commRelationMatrix: ",commRelationMatrix
        print "entityCommOutboundEdges: ",entityCommOutboundEdges
        print "entityCommInboundEdges: ",entityCommInboundEdges
        print "relationCommOutboundEdges: ",relationCommOutboundEdges
        print "relationCommInboundEdges: ",relationCommInboundEdges
        print "commsByEntity: ",commsByEntity
        print "commsByRelation: ",commsByRelation
    print "bestStateTotalPenalty: ", bestStateTotalPenalty
    print "bestStateCommsByEntity: ", bestStateCommsByEntity
    print "bestStateCommsByRelation: ", bestStateCommsByRelation

curEntityNode:  0
outboundEdges:  [ 1.  0.]
inboundEdges:  [ 0.  0.]
curComm:  0
curCommOutboundEdges:  [ 3.  1.]
curCommInboundEdges:  [ 3.  0.]
numNodesInComm:  4.0
outboundDiff:  0.125
inboundDiff:  0.5625
curPenalty 0.6875
curEntityNode:  1
outboundEdges:  [ 1.  0.]
inboundEdges:  [ 0.  0.]
curComm:  0
curCommOutboundEdges:  [ 3.  1.]
curCommInboundEdges:  [ 3.  0.]
numNodesInComm:  4.0
outboundDiff:  0.125
inboundDiff:  0.5625
curPenalty 0.6875
curEntityNode:  2
outboundEdges:  [ 0.  1.]
inboundEdges:  [ 0.  0.]
curComm:  1
curCommOutboundEdges:  [ 0.  1.]
curCommInboundEdges:  [ 0.  2.]
numNodesInComm:  2.0
outboundDiff:  0.25
inboundDiff:  1.0
curPenalty 1.25
curEntityNode:  3
outboundEdges:  [ 1.  0.]
inboundEdges:  [ 1.  0.]
curComm:  0
curCommOutboundEdges:  [ 3.  1.]
curCommInboundEdges:  [ 3.  0.]
numNodesInComm:  4.0
outboundDiff:  0.125
inboundDiff:  0.0625
curPenalty 0.1875
curEntityNode:  4
outboundEdges:  [ 0.  1.]
inboundEdges:  [ 2.  0.]
curComm:  0
curCommOutboundEd